### 3. Analýza známek u studijní skupiny (gql_ug + gql_granting).

Jako parametr vezme definici kontingenční tabulky, GQL dotaz, dotaz zrealizuje, a data vloží do sešitu Excel spolu s kontingenční tabulkou. Vytvořte jako FastAPI službu.

Společné podmínky
- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf
- Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

In [43]:
import aiohttp
import asyncio
import json

In [44]:
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3" #"http://host.docker.internal:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            # print(resp.status)
            keyJson = await resp.json()
            # print(keyJson)
        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            # print(resp.status)
            tokenJson = await resp.json()
            # print(tokenJson)
    return tokenJson.get("token", None)

def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql" #"http://host.docker.internal:33001/api/gql"
        payload = {"query": q, "variables": variables}
        # headers = {"Authorization": f"Bearer {token}"}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            # print(headers, cookies)
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                # print(resp.status)
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

In [46]:
import asyncio

username = "john.newbie@world.com"
password = "john.newbie@world.com"


### TODO:

queryStr = """
{
  result: groupById(id: "2d9dcd22-a4a2-11ed-b9df-0242ac120003") {
    name
    id
    memberships {
      user {
        id
        fullname
        classifications {
          level {
            id
            name
          }
          id
          order
          semester {
            id
            order
            subject {
              id
              name
            }
          }
        }
      }
    }
  }
}
"""

import pandas as pd
def toTable(data):
    return pd.DataFrame(data)

async def fullPipe():
    global pandasData
    
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})
    
    data = response.get("data", None)
    result = data.get("result", None)
    #print(result)

    def convertLevel(membership):
        user = membership["user"]
        classifications = user["classifications"]

        rows = []
        for classification in classifications:
            row = {}
            row["user_fullname"] = membership["user"]["fullname"]
            row["user_id"] = user["id"]

            row["level_id"] = classification["level"]["id"]
            row["level_name"] = classification["level"]["name"]
            row["level_order"] = classification["order"]

            row["subject_id"] = classification["semester"]["subject"]["id"]
            row["subject_name"] = classification["semester"]["subject"]["name"]

            row["semester_id"] = classification["semester"]["id"]
            row["semester_order"] = classification["semester"]["order"]
            rows.append(row)    
        return rows
    
    
    memberships = result["memberships"]
    converted_data = [convertLevel(membership) for membership in memberships]
    
    flattened_data = [item for sublist in converted_data for item in sublist]

    sorted_data = sorted(flattened_data, key=lambda x: x['level_order'], reverse=True)

    filtered_data = sorted_data[:1]
    
    with open('result.json', 'w') as f:
        json.dump(filtered_data, f)
    
    return print(filtered_data)


await fullPipe()

[{'user_fullname': 'John Newbie', 'user_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programování', 'semester_id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'semester_order': 1}]


In [47]:
import pandas as pd
from IPython.display import display

with open('result.json', 'r') as file:
    result = json.load(file)
    
df = pd.DataFrame(result)

grade_mapping = {'A': 1, 'B': 1.5, 'C': 2, 'D': 2.5, 'F': 3}

df['grade_value'] = df['level_name'].map(grade_mapping)

pivot_table = pd.pivot_table(df, 
                             values='grade_value', 
                             index='subject_name', 
                             aggfunc='mean', 
                             fill_value=0)

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

display(pivot_table)


#df = pd.DataFrame(result) # grouped_data


,grade_value
subject_name,
Programování,2


In [42]:
import json

# Load real data
with open('data2.json', 'r', encoding='ISO-8859-1') as file:
    real_data = json.load(file)

# Load fake data
with open('fake_data.json', 'r', encoding='ISO-8859-1') as file:
    fake_data = json.load(file)

# Merge memberships
real_data['data']['groupById']['memberships'].extend(fake_data['data']['groupById']['memberships'])

# Save merged data
with open('merged_data.json', 'w', encoding='utf-8') as file:
    json.dump(real_data, file, indent=4, ensure_ascii=False)